In [6]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

Using TensorFlow backend.


<class 'keras.engine.topology.Layer'>


In [7]:
model = InceptionResNetV2(include_top=True, weights='imagenet', input_tensor=None, input_shape=None)


 35520512/225209952 [===>..........................] - ETA: 1:39


 51314688/225209952 [=====>........................] - ETA: 2:10


 67239936/225209952 [=======>......................] - ETA: 2:26


 78905344/225209952 [=========>....................] - ETA: 2:37


111493120/225209952 [=============>................] - ETA: 1:43


...

153042944/225209952 [===================>..........] - ETA: 1:04


180748288/225209952 [=======================>......] - ETA: 37s


199491584/225209952 [=========================>....] - ETA: 22s


214433792/225209952 [===========================>..] - ETA: 10s


225214464/225209952 [==============================] - 205s   


In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

__________________________________________________________________________________________________
activation_95 (Activation)      (None, None, None, 1 0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_93 (Conv2D)              (None, None, None, 1 208896      block17_4_ac[0][0]               
__________________________________________________________________________________________________
conv2d_96 (Conv2D)              (None, None, None, 1 215040      activation_95[0][0]              
__________________________________________________________________________________________________
batch_normalization_93 (BatchNo (None, None, None, 1 576         conv2d_93[0][0]                  
__________________________________________________________________________________________________
batch_normalization_96 (BatchNo (None, None, None, 1 576         conv2d_96[0][0]                  
__________

In [10]:
import cv2
import keras
from keras.applications.imagenet_utils import preprocess_input
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
from scipy.misc import imread
import tensorflow as tf

from ssd import SSD300
from ssd_utils import BBoxUtility
# coding:utf-8

import keras
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, list_pictures, load_img
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from keras import backend as K
import os
import glob
import matplotlib.gridspec as gridspec
import shutil
from keras.utils import plot_model
import cv2
from skimage import io
from skimage import transform
from scipy.misc import imresize
from skimage import io
import skimage
# フォルダの中にある画像を順次読み込む
# カテゴリーは0から始める

#参考
#https://qiita.com/supersaiakujin/items/fc54116df9ca6958a68d
#http://testpy.hatenablog.com/entry/2017/06/02/001901
#https://qiita.com/zaburo/items/0b9db87d0a52191b164b
# 画像の反転と回転の実装、またはdata argumentation の実装を行う
X = []
Y = []
name_class = []
#下、後でコード量を減らす
# 対象Aの画像
rate = 0.2
label = 0
for dir in os.listdir("data/train"):
#     print(dir)
    if dir == ".DS_Store":
        continue
    name_class.append(dir)
    for picture in list_pictures("data/train/"+dir):
#         print(picture)
    #     img =load_img(picture, target_size=(64,64))
        img =imresize(skimage.io.imread(picture),(224,224), interp='nearest')
#         img =imresize(skimage.io.imread(picture),(64,64), interp='nearest')
    #     img_raw = skimage.io.imread
#         print(img_to_array(skimage.transform.rotate(img, angle=0, resize=False, center=None)).shape)
        for i in range(358,360):
            #回転のみ
            X.append(img_to_array(skimage.transform.rotate(img, angle=i, resize=False, center=None)))
            Y.append(label )
#             #回転と上下の反転
            X.append(img_to_array(cv2.flip(skimage.transform.rotate(img, angle=i, resize=False, center=None),0)))
            Y.append(label )
#             #回転と左右の反転
#             X.append(img_to_array(cv2.flip(skimage.transform.rotate(img, angle=i, resize=False, center=None),1)))
#             Y.append(label )
#             #拡大
            X.append(img_to_array(skimage.transform.warp(skimage.transform.rotate(img, angle=i, resize=False, center=None), skimage.transform.AffineTransform(scale=(1-rate, 1-rate), translation=(64,64)))))
            Y.append(label )
    label += 1

# # arrayに変換
# X.shape()
print(name_class)
X = np.asarray(X)
print(X.shape)
# print(X)
Y = np.asarray(Y)
# print(Y)

# 画素値を0から1の範囲に変換
X = X.astype('float32')
X = X / 255.0

# クラスの形式を変換
Y = np_utils.to_categorical(Y, 5)

# 学習用データとテストデータ
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=111)

['CurryPowder', 'rice', 'onion', 'potato', 'carot']
(1152, 224, 224, 3)


In [27]:
# Finetuning on another 100-class dataset
from keras.applications.inception_resnet_v2 import InceptionResNetV2
base_model = InceptionResNetV2(include_top=False, pooling='avg')
outputs = Dense(label, activation='softmax')(model.output)
new_model = Model(base_model.inputs, outputs)
new_model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
new_model.fit(x_train, y_train, epochs=5, batch_size=32)


  4546560/219055592 [..............................] - ETA: 13:07


  8044544/219055592 [>.............................] - ETA: 13:25


 10813440/219055592 [>.............................] - ETA: 15:29


 15482880/219055592 [=>............................] - ETA: 13:00


 19218432/219055592 [=>............................] - ETA: 12:09


...

182214656/219055592 [=======================>......] - ETA: 57s


197025792/219055592 [=========================>....] - ETA: 33s


211091456/219055592 [===========================>..] - ETA: 11s


219062272/219055592 [==============================] - 309s   


NameError: name 'Dense' is not defined